In [2]:
import PyPDF2
import pandas as pd
import re

In [3]:
def read_pdf(filename):
    pdfFileObj = open(filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    extracted_text=''
    for i in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(i)
        extracted_text+=pageObj.extractText()
    pdfFileObj.close()
    return extracted_text

In [4]:
dataset = pd.read_excel('Sampel Final.xlsx', index_col=0)

In [5]:
dataset.head()

,Nomor Putusan,Jenis Pajak,Tahun Pajak,DV_Menang
No,,,,
1,Put.072196.16.pdf,PPN dan PPnBM,2007,0
2,Put.086980.15.pdf,PPh Badan,2011,0
3,Put.087101.15.pdf,PPh Badan,2012,0
4,Put.098444.15.pdf,PPh Badan,2012,1
5,Put.098538.15.pdf,PPh Badan,2012,0


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 1 to 84
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Nomor Putusan  84 non-null     object
 1   Jenis Pajak    84 non-null     object
 2   Tahun Pajak    84 non-null     int64 
 3   DV_Menang      84 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 3.3+ KB


In [7]:
dataset['pdf_content'] = dataset['Nomor Putusan'].apply(read_pdf)

C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'1' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'2' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'3' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'30' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'33' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in ob

C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'63' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'66' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'65' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'69' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in object header b'68' b'0'
  PdfReadWarning,
C:\ProgramData\Anaconda3\envs\textMining\lib\site-packages\PyPDF2\_reader.py:1197: PdfReadWarning: Superfluous whitespace found in

In [8]:
dataset.head()

,Nomor Putusan,Jenis Pajak,Tahun Pajak,DV_Menang,pdf_content
No,,,,,
1,Put.072196.16.pdf,PPN dan PPnBM,2007,0,TaxBase 6.0 Document - Page : 1Putusan Nomor :...
2,Put.086980.15.pdf,PPh Badan,2011,0,TaxBase 6.0 Document - Page : 1Putusan Pengadi...
3,Put.087101.15.pdf,PPh Badan,2012,0,TaxBase 6.0 Document - Page : 1Putusan Pengadi...
4,Put.098444.15.pdf,PPh Badan,2012,1,TaxBase 6.0 Document - Page : 1Putusan Nomor :...
5,Put.098538.15.pdf,PPh Badan,2012,0,TaxBase 6.0 Document - Page : 1Putusan Nomor :...


In [16]:
#ekstrak pokok sengketa
def extract_circumstances(case_text):
    case_text = re.sub(' +', ' ', case_text)
    case_text = case_text.lower()
    keyword_start = 'pokok sengketa : '
    keyword_limit = 'menurut terbanding : '
    text_result = case_text[case_text.find(keyword_start)+len(keyword_start):case_text.find(keyword_limit)]
    content = text_result.replace('\n',' ')
    return content

#ekstrak pendapat pemerintah (menurut terbanding)
def extract_government_statement(case_text):
    case_text = re.sub(' +', ' ', case_text)
    case_text = case_text.lower()
    keyword_start = 'menurut terbanding : '
    keyword_limit = 'menurut pemohon : '
    keyword_limit_alt = 'menurut pemohon banding : '
    text_result = case_text[case_text.find(keyword_start)+len(keyword_start):case_text.find(keyword_limit)]
    if(case_text.find(keyword_limit)==-1):
        text_result = case_text[case_text.find(keyword_start)+len(keyword_start):case_text.find(keyword_limit_alt)]
        content = text_result.replace('\n',' ')
        return content
    content = text_result.replace('\n',' ')
    return content

#ekstrak pendapat wajib pajak (menurut pemohon)
def extract_taxpayer_statement(case_text):
    case_text = re.sub(' +', ' ', case_text)
    case_text = case_text.lower()
    keyword_start = 'menurut pemohon : '
    keyword_start_alt = 'menurut pemohon banding : '
    keyword_limit = 'menurut majelis : '
    text_result = case_text[case_text.find(keyword_start)+len(keyword_start):case_text.find(keyword_limit)]
    if(case_text.find(keyword_start)==-1):
        text_result = case_text[case_text.find(keyword_start_alt)+len(keyword_start_alt):case_text.find(keyword_limit)]
        content = text_result.replace('\n',' ')
        return content
    content = text_result.replace('\n',' ')
    return content

#ekstrak pendapat hakim pengadilan (menurut majelis)
def extract_court_statement(case_text):
    case_text = re.sub(' +', ' ', case_text)
    case_text = case_text.lower()
    keyword_start = 'menurut majelis : '
    keyword_limit = 'menimbang : '
    text_result = case_text[case_text.find(keyword_start):case_text.find(keyword_limit)]
    content = text_result.replace('\n',' ')
    return content

#ekstrak konsideran (menimbang)
def extract_considerants(case_text):
    case_text = re.sub(' +', ' ', case_text)
    case_text = case_text.lower()
    keyword_start = 'menimbang : '
    keyword_limit = 'mengingat : '
    text_result = case_text[case_text.find(keyword_start)+len(keyword_start):case_text.find(keyword_limit)]
    content = text_result.replace('\n',' ')
    content = content.replace(keyword_start,' ')
    return content

def extract_contents(case_text):
    case_text = re.sub(' +', ' ', case_text)
    case_text = case_text.lower()
    keyword_start = 'pokok sengketa : '
    keyword_limit = 'mengingat : '
    text_result = case_text[case_text.find(keyword_start)+len(keyword_start):case_text.find(keyword_limit)]
    content = text_result.replace('\n',' ')
    return content

def extract_main_contents(case_text):
    case_text = re.sub(' +', ' ', case_text)
    case_text = case_text.lower()
    keyword_start = 'menurut terbanding : '
    keyword_limit = 'menimbang : '
    text_result = case_text[case_text.find(keyword_start)+len(keyword_start):case_text.find(keyword_limit)]
    content = text_result.replace('\n',' ')
    return content

In [10]:
#clean taxbase header
def clean_taxbase_header(text_to_clean):
    tb = 'TaxBase 6.0 Document - Page : '
    text = text_to_clean
    for i in range(60):        
        text = text.replace(tb+str(i+1),' ')
    return text

In [11]:
dataset['pdf_content_cleaned'] = dataset.pdf_content.apply(clean_taxbase_header)

In [17]:
dataset['case_circumstances'] = dataset.pdf_content_cleaned.apply(extract_circumstances)
dataset['government_statement'] = dataset.pdf_content_cleaned.apply(extract_government_statement)
dataset['taxpayer_statement'] = dataset.pdf_content_cleaned.apply(extract_taxpayer_statement)
dataset['court_statement'] = dataset.pdf_content_cleaned.apply(extract_court_statement)
dataset['court_considerants'] = dataset.pdf_content_cleaned.apply(extract_considerants)
dataset['case_contents'] = dataset.pdf_content_cleaned.apply(extract_contents)
dataset['case_main_contents'] = dataset.pdf_content_cleaned.apply(extract_main_contents)

In [18]:
dataset.head(56)

,Nomor Putusan,Jenis Pajak,Tahun Pajak,DV_Menang,pdf_content,pdf_content_cleaned,case_circumstances,government_statement,taxpayer_statement,court_statement,court_considerants,case_contents,case_main_contents
No,,,,,,,,,,,,,
1,Put.072196.16.pdf,PPN dan PPnBM,2007,0,TaxBase 6.0 Document - Page : 1Putusan Nomor :...,Putusan Nomor : PUT-072196.16/2007/PP/M.VB Ta...,bahwa nilai sengketa dalam sengketa banding in...,bahwa terbanding melakukan koreksi atas transa...,bahwa bukti yang pemohon banding sampaikan jel...,menurut majelis : bahwa yang menjadi sengketa ...,bahwa dalam sengketa banding ini tidak terdapa...,bahwa nilai sengketa dalam sengketa banding in...,bahwa terbanding melakukan koreksi atas transa...
2,Put.086980.15.pdf,PPh Badan,2011,0,TaxBase 6.0 Document - Page : 1Putusan Pengadi...,Putusan Pengadilan Pajak Nomor : PUT-086980.1...,bahwa nilai sengketa terbukti dalam sengketa b...,a. koreksi atas biaya royalty sebesar rp. 52.0...,menurut pemohon banding koreksi atas pembayara...,menurut majelis : bahwa yang menjadi sengketa ...,bahwa dalam sengketa banding ini tidak terdapa...,bahwa nilai sengketa terbukti dalam sengketa b...,a. koreksi atas biaya royalty sebesar rp. 52.0...
3,Put.087101.15.pdf,PPh Badan,2012,0,TaxBase 6.0 Document - Page : 1Putusan Pengadi...,Putusan Pengadilan Pajak Nomor : Put-087101.1...,bahwa nilai sengketa terbukti dalam sengketa b...,bahwa menurut terbanding koreksi harga pokok p...,bahwa metode comparable uncontrolled price (cu...,menurut majelis : bahwa majelis berpendapat sk...,bahwa dalam hal dilakukan analisa berdasar com...,bahwa nilai sengketa terbukti dalam sengketa b...,bahwa menurut terbanding koreksi harga pokok p...
4,Put.098444.15.pdf,PPh Badan,2012,1,TaxBase 6.0 Document - Page : 1Putusan Nomor :...,Putusan Nomor : PUT-098444.15/2012/PP/M.VIIIB...,bahwa nilai sengketa terbukti dalam banding in...,bahwa dasar dilakukan koreksi technical assist...,pemohon banding mengemukakan mengenai jasa yan...,menurut majelis : bahwa yang menjadi sengketa ...,"atas hasil pemeriksaan dalam persidangan, maje...",bahwa nilai sengketa terbukti dalam banding in...,bahwa dasar dilakukan koreksi technical assist...
5,Put.098538.15.pdf,PPh Badan,2012,0,TaxBase 6.0 Document - Page : 1Putusan Nomor :...,Putusan Nomor : PUT-098538.15/2012/PP/M.XB Ta...,bahwa nilai yang menjadi pokok sengketa dalam ...,bahwa oleh terbanding dalam sub dan penjelasan...,bahwa pemohon banding menambahkan penjelasanny...,menurut majelis : bahwa yang menjadi sengketa ...,bahwa atas hasil pemeriksaan dalam persidangan...,bahwa nilai yang menjadi pokok sengketa dalam ...,bahwa oleh terbanding dalam sub dan penjelasan...
6,Put.101125.16.pdf,PPN dan PPnBM,2012,0,TaxBase 6.0 Document - Page : 1Putusan Pengadi...,Putusan Pengadilan Pajak Nomor : PUT-101125.1...,bahwa nilai sengketa terbukti dalam sengketa b...,bahwa koreksi terbanding telah sesuai dengan k...,bahwa pemohon banding tidak setuju dengan alas...,menurut majelis : bahwa berdasarkan pemeriksaa...,"bahwa berdasarkan uraian tersebut di atas, rek...",bahwa nilai sengketa terbukti dalam sengketa b...,bahwa koreksi terbanding telah sesuai dengan k...
7,Put.101126.16.pdf,PPN dan PPnBM,2012,0,TaxBase 6.0 Document - Page : 1Putusan Pengadi...,Putusan Pengadilan Pajak Nomor : PUT-101126.1...,bahwa nilai sengketa terbukti dalam sengketa b...,bahwa koreksi pemeriksa telah sesuai dengan ke...,bahwa pemohon banding tidak setuju dengan alas...,menurut majelis : bahwa berdasarkan pemeriksaa...,"bahwa berdasarkan uraian tersebut di atas, rek...",bahwa nilai sengketa terbukti dalam sengketa b...,bahwa koreksi pemeriksa telah sesuai dengan ke...
8,Put.101736.16.pdf,PPN dan PPnBM,2011,0,TaxBase 6.0 Document - Page : 1Putusan Nomor :...,Putusan Nomor : 101736.16/2011/PP/M.IA Tahun ...,koreksi pajak masukan masa pajak november 2011...,bahwa berdasarkan lhp dan kkp dijelaskan bahwa...,bahwa pemohon banding tidak setuju dengan kore...,menurut majelis : bahwa yang menjadi sengketa ...,bahwa atas hasil

In [19]:
data = dataset.loc[:, dataset.columns != 'pdf_content']
data = data.loc[:, data.columns != 'pdf_content_cleaned']
data.head()

,Nomor Putusan,Jenis Pajak,Tahun Pajak,DV_Menang,case_circumstances,government_statement,taxpayer_statement,court_statement,court_considerants,case_contents,case_main_contents
No,,,,,,,,,,,
1,Put.072196.16.pdf,PPN dan PPnBM,2007,0,bahwa nilai sengketa dalam sengketa banding in...,bahwa terbanding melakukan koreksi atas transa...,bahwa bukti yang pemohon banding sampaikan jel...,menurut majelis : bahwa yang menjadi sengketa ...,bahwa dalam sengketa banding ini tidak terdapa...,bahwa nilai sengketa dalam sengketa banding in...,bahwa terbanding melakukan koreksi atas transa...
2,Put.086980.15.pdf,PPh Badan,2011,0,bahwa nilai sengketa terbukti dalam sengketa b...,a. koreksi atas biaya royalty sebesar rp. 52.0...,menurut pemohon banding koreksi atas pembayara...,menurut majelis : bahwa yang menjadi sengketa ...,bahwa dalam sengketa banding ini tidak terdapa...,bahwa nilai sengketa terbukti dalam sengketa b...,a. koreksi atas biaya royalty sebesar rp. 52.0...
3,Put.087101.15.pdf,PPh Badan,2012,0,bahwa nilai sengketa terbukti dalam sengketa b...,bahwa menurut terbanding koreksi harga pokok p...,bahwa metode comparable uncontrolled price (cu...,menurut majelis : bahwa majelis berpendapat sk...,bahwa dalam hal dilakukan analisa berdasar com...,bahwa nilai sengketa terbukti dalam sengketa b...,bahwa menurut terbanding koreksi harga pokok p...
4,Put.098444.15.pdf,PPh Badan,2012,1,bahwa nilai sengketa terbukti dalam banding in...,bahwa dasar dilakukan koreksi technical assist...,pemohon banding mengemukakan mengenai jasa yan...,menurut majelis : bahwa yang menjadi sengketa ...,"atas hasil pemeriksaan dalam persidangan, maje...",bahwa nilai sengketa terbukti dalam banding in...,bahwa dasar dilakukan koreksi technical assist...
5,Put.098538.15.pdf,PPh Badan,2012,0,bahwa nilai yang menjadi pokok sengketa dalam ...,bahwa oleh terbanding dalam sub dan penjelasan...,bahwa pemohon banding menambahkan penjelasanny...,menurut majelis : bahwa yang menjadi sengketa ...,bahwa atas hasil pemeriksaan dalam persidangan...,bahwa nilai yang menjadi pokok sengketa dalam ...,bahwa oleh terbanding dalam sub dan penjelasan...


In [20]:
data.to_csv('tax_court_analytics_data.csv',index=False)